In [1]:
import requests
import pandas as pd
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output  # pip install dash (version 2.0.0 or higher)
import json

In [ ]:
!pip install dash

In [2]:
url = "https://goverlytics.p.rapidapi.com/federal-legislation/ca"

gender_list = ["M", "F", "O"]

In [3]:
def load_data(limit, page, gender):

    querystring = {"include_summary":"true",
                   "include_text":"true",
                   "include_actions":"true",
                   "include_votes":"true",
                   "sponsor_gender": gender,
                   "limit": limit,
                   "page": page,
                  }

    headers = {
        'x-rapidapi-host': "goverlytics.p.rapidapi.com",
        'x-rapidapi-key': "4d08b76ec0msh553b06f36bc2c52p18a08ejsne688f7dc5de5"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    return response


def load_large_data(pages):
    list_df = []
    for page in range(pages):
        for gender in gender_list:
            list_df.append(pd.DataFrame(load_data(25, page, gender).json()["data"]))
            
    df = pd.concat(list_df, axis=0)
    print(df.shape)

    return df

In [4]:
list_provinces = ['AB', 'BC', 'MB', 'NB', 'NL', 'NT', 'NS', 'NU', 'ON', 'PE', 'QC', 'SK', 'YT', None]
name_provinces = ["Alberta", "British Columbia", "Manitoba", "New Brunswick", "Newfoundland and Labrador",
                  "Northwest Territories", "Nova Scotia", "Nunavut", "Ontario", "Prince Edward Island",
                  "Quebec", "Saskatchewan", "Yukon", None]

canada_province = json.load(open("../input/canada-provinces/canada_provinces.geojson", "r"))

In [5]:
def simplified_df(df):
    new_df = df[['province_territory', 'last_major_event', 'topic']].copy()
    new_df['last_status'] = 0
    new_df['final_year'] = 0

    for index, row in new_df.iterrows():
        new_df.loc[index, 'last_status'] = row['last_major_event']['status']
        # new_df.at[index,'last_status'] = row['last_major_event']['status']
        new_df.loc[index, 'final_year'] = row['last_major_event']['date'].split('-')[0]

    new_df = new_df.drop('last_major_event', axis=1)
    
    topic_df = new_df.groupby(['topic']).size()
    topic_df = topic_df.reset_index().drop([0]).reset_index().drop(['index'], axis=1).rename({0: 'count'}, axis=1)\
                        .sort_values(by=['count'])
    
    return new_df, topic_df


def passed_bills(df):

    new_df, topic_df = simplified_df(df)

    passed_bills_df = pd.DataFrame(data={"name": name_provinces, "pr_ter": list_provinces})

    passed_bills_df["topic"] = None
    passed_bills_df["2019"] = 0
    passed_bills_df["2020"] = 0
    passed_bills_df["2021"] = 0

    passed_bills_df = passed_bills_df.set_index('pr_ter')

    for index, row in new_df.iterrows():
        if row['last_status'] == 'Royal Assent':
            passed_bills_df.loc[row['province_territory'], row['final_year']] += 1

        passed_bills_df.loc[row['province_territory'], "topic"] = row['topic']

    passed_bills_df = passed_bills_df.reset_index()
    passed_bills_df['total'] = passed_bills_df[['2019', '2020', '2021']].sum(axis=1)

    return passed_bills_df, topic_df


def update_df(old_df):

    df, topic_df = passed_bills(old_df)

    province_id_map = {}
    for feature in canada_province["features"]:
        feature['id'] = feature['properties']['cartodb_id']
        province_id_map[feature['properties']['name']] = feature['id']

    province_id_map[None] = None

    print(province_id_map)

    df['id'] = df['name'].apply(lambda x: province_id_map[x])

    return df, topic_df


def topic_bar_plot(df):
    fig = px.bar(df, x='topic', y='count',
             #hover_data=['lifeExp', 'gdpPercap'], 
             color='count',
             #labels={'pop':'population of Canada'}, 
             height=400,
             title="Topics of passed bills")
    
    return fig


def draw_map(year, df):
    fig = px.choropleth(df,
                        locations='id',
                        geojson=canada_province,
                        color=str(year),
                        )

    return fig

In [6]:
app = Dash(__name__)

# -- Import and clean data (importing csv into pandas)

df = load_large_data(10)
df, topic_df = update_df(df)

In [ ]:
# App layout
app.layout = html.Div([

    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),

    dcc.Dropdown(id="slct_year",
                 options=[
                     {"label": "2021", "value": 2021},
                     {"label": "2019", "value": 2019},
                     {"label": "2020", "value": 2020},],
                 multi=False,
                 value=2021,
                 style={'width': "40%"}
                 ),

    html.Div(id='output_container', children=[]),
    html.Br(),
    
    dcc.Graph(id='my_canada_map', figure={}),
    
    dcc.Graph(id='my_bar_plot', figure={})

])

html.Div(className='row', children=[
    html.H1("Tips database analysis (First dashboard)"),
    dcc.Dropdown(),
    html.Div(children=[
        dcc.Graph(id="graph1", style={'display': 'inline-block'}),
        dcc.Graph(id="graph2", style={'display': 'inline-block'})
    ])
])

# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_canada_map', component_property='figure'),
     Output(component_id='my_bar_plot', component_property='figure')],
    [Input(component_id='slct_year', component_property='value')]
)
def update_graph(option_slctd):
    print(option_slctd)
    print(type(option_slctd))

    container = "The year chosen by user was: {}".format(option_slctd)

    fig1 = draw_map(option_slctd, df)
    fig2 = topic_bar_plot(topic_df)

    return container, fig1, fig2


# -----------------------------------------------------------------------------
if __name__ == '__main__':
    app.run_server(debug=False)